<a href="https://colab.research.google.com/github/11bender/alumni-scraping/blob/main/alumni_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Twitter scraper**

## Load libraries

In [1]:
import os, random, sys, time 
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np 

## Get Tweet URL

In [ ]:
driver = webdriver.Chrome('/driver/chromedriver')
driver.get("https://www.twitter.com/login/")

- Hashtag: #ARGFRA OR #WORLDCUPFINAL OR #argentinavsfrance
- Time: until:2022-12-20 since:2022-12-16
- Language: lang:en
- Geocoding: geocode:54.2361,-2.5487,250mi (USA)



-is:retweet #ARGFRA OR #WORLDCUPFINAL geocode:54.2361,-2.5487,250mi until:2022-12-19 since:2022-12-17 lang:en

In [ ]:
# SEARCH QUERY
url = "https://twitter.com/search?q=-is%3Aretweet%20%23ARGFRA%20OR%20%23WORLDCUPFINAL%20until%3A2022-12-18%20since%3A2022-12-17%20lang%3Aen%20geocode%3A52.3555%2C-1.1743%2C250km&src=typed_query&f=top"
driver.get(url)
# Initiate a list
url_lst = []

In [ ]:
current_height = driver.execute_script("return document.body.scrollHeight")
stops = 0
MAX_STOPS = 3

while True:
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
        
    urls = soup.find_all('div', {'class': "css-1dbjc4n r-18u37iz r-1q142lx"})
    for url in urls:
        try: 
            clean = url.find('a', {'class': 'css-4rbku5'})['href']
            url_lst.append(clean)
        except TypeError:
            continue

    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    
    new_height = driver.execute_script("return document.body.scrollHeight")
        
    if new_height == current_height:
        stops += 1
        if stops == MAX_STOPS:
            break
        time.sleep(10)
    else:
        current_height = new_height
        stops = 0

In [ ]:
# Remove duplicates
url_lst = list(set(url_lst))
# Export to csv
# Change name accordingly
# england_url = pd.DataFrame(url_lst, columns=['url'])
# england_url.to_csv('england_url.csv')

## Scrape tweets

In [ ]:
urls = []
content_lst = []
timestamp_lst = []
emoji_lst = []
reactions_lst = []
is_retweeted = []
is_quoted = []
is_liked = []
is_bookmarked = []

twitter_url = "https://www.twitter.com"

for i, url in enumerate(url_lst):
  tweet_url = twitter_url + url
  urls.append(tweet_url)
  print((i+1)/len(url_lst))
  driver.get(tweet_url)
  time.sleep(5)
  src = driver.page_source
  soup = BeautifulSoup(src, 'html.parser')
  
  # scrape content
  content = soup.find('div', {"data-testid":"tweetText"}).get_text()
  content_lst.append(content)
  # scrape timestamp
  timestamp_lst.append(soup.find('time')['datetime'])
  # scrape emojis
  emojis = soup.find('div', {'class': "css-901oao r-18jsvk2 r-37j5jr r-1inkyih r-16dba41 r-rjixqe r-bcqeeo r-bnwqim r-qvutc0"})
  emoji_lst.append(list(set([emo.get('src') for emo in emojis.find_all('img')])))
  # scrape reactions
  reaction = soup.find_all('span', {'class': "css-901oao css-16my406 r-poiln3 r-1b43r93 r-b88u0q r-1cwl3u0 r-bcqeeo r-qvutc0"})
  reactions = [r.get_text() for r in reaction]
  reactions_lst.append(reactions)
  # has View or Retweet?
  misc = soup.find_all('span', {'class': "css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0"})
  stats = [m.get_text() for m in misc]
  is_retweeted.append(1 if "Retweet" in stats else 0)
  is_quoted.append(1 if "Quote" in stats else 0)
  is_liked.append(1 if "Likes" in stats else 0)
  is_bookmarked.append(1 if "Bookmark" in stats else 0)
  


In [ ]:
data = pd.DataFrame(list(zip(urls, content_lst, emoji_lst, timestamp_lst, reactions_lst, is_retweeted, is_quoted, is_liked, is_bookmarked)), 
               columns =['url', 'tweet', 'emoji', 'timestamp', 'reactions', 'is_retweeted', 'is_quoted', 'is_liked', 'is_bookmarked']) 

In [ ]:
data.to_csv('xxx.csv', index=False)